### Seq2seq

Un modelo secuencia a secuencia (Seq2Seq) es una arquitectura de red neuronal diseñada para transformar una secuencia de entrada en una secuencia de salida, y es especialmente útil para tareas donde la longitud de las secuencias de entrada y salida puede diferir. Los modelos Seq2Seq son comúnmente utilizados en aplicaciones como traducción automática, resumen de texto, y generación de texto.

In [ ]:
import numpy as np
import torch
import torch.nn as nn

# S: Símbolo que muestra el inicio de la entrada de decodificación
# E: Símbolo que muestra el inicio de la salida de decodificación
# P: Símbolo que llenará la secuencia en blanco si el tamaño de los datos del lote actual es menor que los pasos de tiempo

def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        input_seq = [num_dic[n] for n in seq[0]]
        output_seq = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(n_class)[input_seq])
        output_batch.append(np.eye(n_class)[output_seq])
        target_batch.append(target) # no es one-hot

    # Convertir listas a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    # crear tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# crear lote de prueba
def make_testbatch(input_word):
    input_batch, output_batch = [], []

    input_w = input_word + 'P' * (n_step - len(input_word))
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    # Convertir a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Modelo
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1) # enc_input: [max_len(=n_step, pasos de tiempo), tamaño_lote, n_class]
        dec_input = dec_input.transpose(0, 1) # dec_input: [max_len(=n_step, pasos de tiempo), tamaño_lote, n_class]

        # enc_states : [num_layers * num_directions(=1), tamaño_lote, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)
        # outputs : [max_len+1(=6), tamaño_lote, num_directions(=1) * n_hidden(=128)]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        model = self.fc(outputs) # model : [max_len+1(=6), tamaño_lote, n_class]
        return model

if __name__ == '__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
    num_dic = {n: i for i, n in enumerate(char_arr)}
    seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

    n_class = len(num_dic)
    tamaño_lote = len(seq_data)

    model = Seq2Seq()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()

    for epoch in range(5000):
        # crear forma oculta [num_layers * num_directions, tamaño_lote, n_hidden]
        hidden = torch.zeros(2, tamaño_lote, n_hidden) # cambiar a 2 capas

        optimizer.zero_grad()
        # input_batch : [tamaño_lote, max_len(=n_step, pasos de tiempo), n_class]
        # output_batch : [tamaño_lote, max_len+1(=n_step, pasos de tiempo) (por 'S' o 'E'), n_class]
        # target_batch : [tamaño_lote, max_len+1(=n_step, pasos de tiempo)], no es one-hot
        output = model(input_batch, hidden, output_batch)
        # output : [max_len+1, tamaño_lote, n_class]
        output = output.transpose(0, 1) # [tamaño_lote, max_len+1(=6), n_class]
        loss = 0
        for i in range(0, len(target_batch)):
            # output[i] : [max_len+1, n_class, target_batch[i] : max_len+1]
            loss += criterion(output[i], target_batch[i])
        if (epoch + 1) % 1000 == 0:
            print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()

    # Prueba
    def translate(word):
        input_batch, output_batch = make_testbatch(word)

        # crear forma oculta [num_layers * num_directions, tamaño_lote, n_hidden]
        hidden = torch.zeros(2, 1, n_hidden) # cambiar a 2 capas
        output = model(input_batch, hidden, output_batch)
        # output : [max_len+1(=6), tamaño_lote(=1), n_class]

        predict = output.data.max(2, keepdim=True)[1] # seleccionar dimensión n_class
        decoded = [char_arr[i] for i in predict]
        end = decoded.index('E')
        translated = ''.join(decoded[:end])

        return translated.replace('P', '')

    print('test')
    print('man ->', translate('man'))
    print('mans ->', translate('mans'))
    print('king ->', translate('king'))
    print('black ->', translate('black'))
    print('upp ->', translate('upp'))


### Greedy, beam search y selección aleatoria con temperatura

Podemos implementar las estrategias de decodificación greedy, beam search y selección aleatoria con temperatura, podemos modificar el código dado agregando funciones específicas para cada método de decodificación.

**Estrategia greedy**

La estrategia Greedy selecciona el token con la mayor probabilidad en cada paso de decodificación.


In [ ]:
import numpy as np
import torch
import torch.nn as nn

# S: Símbolo que muestra el inicio de la entrada de decodificación
# E: Símbolo que muestra el inicio de la salida de decodificación
# P: Símbolo que llenará la secuencia en blanco si el tamaño de los datos del lote actual es menor que los pasos de tiempo

def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        input_seq = [num_dic[n] for n in seq[0]]
        output_seq = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(n_class)[input_seq])
        output_batch.append(np.eye(n_class)[output_seq])
        target_batch.append(target)  # no es one-hot

    # Convertir listas a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    # Crear tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# Crear lote de prueba
def make_testbatch(input_word):
    input_batch, output_batch = [], []

    input_w = input_word + 'P' * (n_step - len(input_word))
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    # Convertir a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Modelo
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1)  # [max_len, tamaño_lote, n_class]
        dec_input = dec_input.transpose(0, 1)  # [max_len, tamaño_lote, n_class]

        # enc_states : [num_layers, tamaño_lote, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)
        # outputs : [max_len, tamaño_lote, n_hidden]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        model = self.fc(outputs)  # [max_len, tamaño_lote, n_class]
        return model, enc_states

if __name__ == '__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
    num_dic = {n: i for i, n in enumerate(char_arr)}
    seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

    n_class = len(num_dic)
    tamaño_lote = len(seq_data)

    model = Seq2Seq()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()

    for epoch in range(5000):
        model.train()
        # Crear forma oculta [num_layers, tamaño_lote, n_hidden]
        hidden = torch.zeros(2, tamaño_lote, n_hidden)  # 2 capas

        optimizer.zero_grad()
        # Forward pass
        output, _ = model(input_batch, hidden, output_batch)
        # output : [max_len, tamaño_lote, n_class]
        output = output.transpose(0, 1)  # [tamaño_lote, max_len, n_class]
        loss = 0
        for i in range(tamaño_lote):
            loss += criterion(output[i], target_batch[i])
        if (epoch + 1) % 1000 == 0:
            print('Época:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss.item()))
        loss.backward()
        optimizer.step()

    # Prueba con decodificación greedy
    def translate(word):
        model.eval()  # Modo evaluación
        with torch.no_grad():  # Desactivar cálculo de gradientes
            input_w = word + 'P' * (n_step - len(word))
            input_seq = [num_dic[n] for n in input_w]
            input_batch = np.eye(n_class)[input_seq]
            input_batch = torch.FloatTensor(input_batch).unsqueeze(0)  # [1, max_len, n_class]

            # Inicializar el estado oculto del codificador
            hidden = torch.zeros(2, 1, n_hidden)  # 2 capas

            # Codificar la entrada
            enc_input = input_batch.transpose(0, 1)  # [max_len, 1, n_class]
            _, enc_states = model.enc_cell(enc_input, hidden)

            # Inicializar la entrada del decodificador con el símbolo de inicio 'S'
            decoder_input = torch.zeros(1, 1, n_class)  # [1, 1, n_class]
            decoder_input[0, 0, num_dic['S']] = 1.0

            decoded = []
            for _ in range(n_step + 1):  # +1 para incluir 'E'
                # Decodificar paso a paso
                output, enc_states = model.dec_cell(decoder_input, enc_states)
                output = model.fc(output.squeeze(0))  # [1, n_class]

                # Seleccionar el token con mayor probabilidad
                _, topi = output.topk(1)  # [1]
                next_token = topi.item()

                # Obtener el carácter correspondiente
                char = char_arr[next_token]
                if char == 'E':
                    break
                if char != 'P':  # Ignorar los rellenos
                    decoded.append(char)

                # Preparar la siguiente entrada del decodificador
                decoder_input = torch.zeros(1, 1, n_class)
                decoder_input[0, 0, next_token] = 1.0

            translated = ''.join(decoded)
            return translated

    print('Prueba de decodificación greedy:')
    print('man ->', translate('man'))
    print('mans ->', translate('mans'))
    print('king ->', translate('king'))
    print('black ->', translate('black'))
    print('upp ->', translate('upp'))


**Estrategia beam search**

Beam Search mantiene las mejores k secuencias en cada paso, lo que permite explorar múltiples caminos en la decodificación y seleccionar la secuencia más probable.

In [ ]:
import numpy as np
import torch
import torch.nn as nn

# S: Símbolo que muestra el inicio de la entrada de decodificación
# E: Símbolo que muestra el fin de la salida de decodificación
# P: Símbolo que llenará la secuencia en blanco si el tamaño de los datos del lote actual es menor que los pasos de tiempo

def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            # Rellenar las secuencias con 'P' para alcanzar n_step
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        input_seq = [num_dic[n] for n in seq[0]]
        output_seq = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(n_class)[input_seq])
        output_batch.append(np.eye(n_class)[output_seq])
        target_batch.append(target)  # No es one-hot

    # Convertir listas a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    # Crear tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# Crear lote de prueba
def make_testbatch(input_word):
    input_batch, output_batch = [], []

    input_w = input_word + 'P' * (n_step - len(input_word))
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    # Convertir a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Modelo
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1)  # [max_len, tamaño_lote, n_class]
        dec_input = dec_input.transpose(0, 1)  # [max_len, tamaño_lote, n_class]

        # enc_states : [num_layers, tamaño_lote, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)
        # outputs : [max_len, tamaño_lote, n_hidden]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        model = self.fc(outputs)  # [max_len, tamaño_lote, n_class]
        return model, enc_states

if __name__ == '__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
    num_dic = {n: i for i, n in enumerate(char_arr)}
    seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

    n_class = len(num_dic)
    tamaño_lote = len(seq_data)

    model = Seq2Seq()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()

    for epoch in range(5000):
        model.train()
        # Crear forma oculta [num_layers, tamaño_lote, n_hidden]
        hidden = torch.zeros(2, tamaño_lote, n_hidden)  # 2 capas

        optimizer.zero_grad()
        # Forward pass
        output, _ = model(input_batch, hidden, output_batch)
        # output : [max_len, tamaño_lote, n_class]
        output = output.transpose(0, 1)  # [tamaño_lote, max_len, n_class]
        loss = 0
        for i in range(tamaño_lote):
            loss += criterion(output[i], target_batch[i])
        if (epoch + 1) % 1000 == 0:
            print('Época:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss.item()))
        loss.backward()
        optimizer.step()

    # Prueba con decodificación beam search
    def translate_beam_search(word, beam_size=3, max_dec_steps=10):
        model.eval()  # Modo evaluación
        with torch.no_grad():  # Desactivar cálculo de gradientes
            input_w = word + 'P' * (n_step - len(word))
            input_seq = [num_dic.get(n, num_dic['P']) for n in input_w]
            input_batch = np.eye(n_class)[input_seq]
            input_batch = torch.FloatTensor(input_batch).unsqueeze(0)  # [1, max_len, n_class]

            # Inicializar el estado oculto del codificador
            hidden = torch.zeros(2, 1, n_hidden)  # 2 capas

            # Codificar la entrada
            enc_input = input_batch.transpose(0, 1)  # [max_len, 1, n_class]
            _, enc_states = model.enc_cell(enc_input, hidden)

            # Inicializar el haz con la secuencia inicial [S]
            beam = [{
                'sequence': [num_dic['S']],
                'hidden': enc_states.clone(),
                'score': 0.0
            }]

            completed_sequences = []

            for _ in range(max_dec_steps):
                new_beam = []
                for seq in beam:
                    last_token = seq['sequence'][-1]
                    if last_token == num_dic['E']:
                        completed_sequences.append(seq)
                        continue

                    # Preparar la entrada del decodificador (one-hot)
                    decoder_input = torch.zeros(1, 1, n_class)
                    decoder_input[0, 0, last_token] = 1.0

                    # Obtener la salida del decodificador
                    dec_output, dec_hidden = model.dec_cell(decoder_input, seq['hidden'])
                    logits = model.fc(dec_output.squeeze(0))  # [1, n_class]
                    log_probs = nn.functional.log_softmax(logits, dim=1)  # [1, n_class]

                    topk_log_probs, topk_indices = torch.topk(log_probs, beam_size, dim=1)  # [1, beam_size]

                    for i in range(beam_size):
                        token = topk_indices[0, i].item()
                        score = seq['score'] + topk_log_probs[0, i].item()
                        new_seq = seq['sequence'] + [token]
                        new_hidden = dec_hidden.clone()
                        new_beam.append({
                            'sequence': new_seq,
                            'hidden': new_hidden,
                            'score': score
                        })

                # Ordenar todas las nuevas secuencias por su puntuación y seleccionar las top `beam_size`
                new_beam = sorted(new_beam, key=lambda x: x['score'], reverse=True)
                beam = new_beam[:beam_size]

                # Si todas las secuencias en el haz han finalizado, detener la búsqueda
                if len(completed_sequences) >= beam_size:
                    break

            # Si no hay secuencias completadas, usar las actuales
            if not completed_sequences:
                completed_sequences = beam

            # Ordenar las secuencias completadas por su puntuación y seleccionar la mejor
            completed_sequences = sorted(completed_sequences, key=lambda x: x['score'], reverse=True)
            best_sequence = completed_sequences[0]['sequence']

            # Convertir la secuencia de índices a caracteres
            decoded = [char_arr[i] for i in best_sequence]

            # Encontrar el índice del símbolo de fin 'E'
            if 'E' in decoded:
                end = decoded.index('E')
                decoded = decoded[:end]

            translated = ''.join([char for char in decoded if char not in ['S', 'P']])

            return translated

    print('\nPrueba de decodificación beam search:')
    print('man ->', translate_beam_search('man'))
    print('mans ->', translate_beam_search('mans'))
    print('king ->', translate_beam_search('king'))
    print('black ->', translate_beam_search('black'))
    print('upp ->', translate_beam_search('upp'))


**Selección aleatoria con temperatura**

La selección aleatoria con temperatura ajusta las probabilidades de los tokens antes de muestrear de la distribución, permitiendo más exploración en la generación.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# S: Símbolo que muestra el inicio de la entrada de decodificación
# E: Símbolo que muestra el inicio de la salida de decodificación
# P: Símbolo que llenará la secuencia en blanco si el tamaño de los datos del lote actual es menor que los pasos de tiempo

def make_batch():
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        for i in range(2):
            seq[i] = seq[i] + 'P' * (n_step - len(seq[i]))

        input_seq = [num_dic[n] for n in seq[0]]
        output_seq = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(n_class)[input_seq])
        output_batch.append(np.eye(n_class)[output_seq])
        target_batch.append(target) # no es one-hot

    # Convertir listas a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)
    target_batch = np.array(target_batch)

    # crear tensor
    return torch.FloatTensor(input_batch), torch.FloatTensor(output_batch), torch.LongTensor(target_batch)

# crear lote de prueba
def make_testbatch(input_word):
    input_batch, output_batch = [], []

    input_w = input_word + 'P' * (n_step - len(input_word))
    input_seq = [num_dic[n] for n in input_w]
    output_seq = [num_dic[n] for n in 'S' + 'P' * n_step]

    input_batch = np.eye(n_class)[input_seq]
    output_batch = np.eye(n_class)[output_seq]

    # Convertir a numpy arrays antes de convertir a tensores
    input_batch = np.array(input_batch)
    output_batch = np.array(output_batch)

    return torch.FloatTensor(input_batch).unsqueeze(0), torch.FloatTensor(output_batch).unsqueeze(0)

# Modelo
class Seq2Seq(nn.Module):
    def __init__(self):
        super(Seq2Seq, self).__init__()

        self.enc_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.dec_cell = nn.RNN(input_size=n_class, hidden_size=n_hidden, num_layers=2, dropout=0.5)
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, enc_input, enc_hidden, dec_input):
        enc_input = enc_input.transpose(0, 1) # enc_input: [max_len(=n_step, pasos de tiempo), tamaño_lote, n_class]
        dec_input = dec_input.transpose(0, 1) # dec_input: [max_len(=n_step, pasos de tiempo), tamaño_lote, n_class]

        # enc_states : [num_layers * num_directions(=1), tamaño_lote, n_hidden]
        _, enc_states = self.enc_cell(enc_input, enc_hidden)
        # outputs : [max_len+1(=6), tamaño_lote, num_directions(=1) * n_hidden(=128)]
        outputs, _ = self.dec_cell(dec_input, enc_states)

        model = self.fc(outputs) # model : [max_len+1(=6), tamaño_lote, n_class]
        return model

if __name__ == '__main__':
    n_step = 5
    n_hidden = 128

    char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
    num_dic = {n: i for i, n in enumerate(char_arr)}
    seq_data = [['man', 'women'], ['black', 'white'], ['king', 'queen'], ['girl', 'boy'], ['up', 'down'], ['high', 'low']]

    n_class = len(num_dic)
    tamaño_lote = len(seq_data)

    model = Seq2Seq()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    input_batch, output_batch, target_batch = make_batch()

    for epoch in range(5000):
        # crear forma oculta [num_layers * num_directions, tamaño_lote, n_hidden]
        hidden = torch.zeros(2, tamaño_lote, n_hidden) # cambiar a 2 capas

        optimizer.zero_grad()
        # input_batch : [tamaño_lote, max_len(=n_step, pasos de tiempo), n_class]
        # output_batch : [tamaño_lote, max_len+1(=n_step, pasos de tiempo) (por 'S' o 'E'), n_class]
        # target_batch : [tamaño_lote, max_len+1(=n_step, pasos de tiempo)], no es one-hot
        output = model(input_batch, hidden, output_batch)
        # output : [max_len+1, tamaño_lote, n_class]
        output = output.transpose(0, 1) # [tamaño_lote, max_len+1(=6), n_class]
        loss = 0
        for i in range(0, len(target_batch)):
            # output[i] : [max_len+1, n_class, target_batch[i] : max_len+1]
            loss += criterion(output[i], target_batch[i])
        if (epoch + 1) % 1000 == 0:
            print('Epoca:', '%04d' % (epoch + 1), 'costo =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()

    # Estrategias de Decodificación
    def greedy_decode(input_batch, hidden, output_batch):
        output = model(input_batch, hidden, output_batch)
        predict = output.data.max(2, keepdim=True)[1] # seleccionar dimensión n_class
        return predict

    def beam_search_decode(input_batch, hidden, output_batch, beam_width=3):
        output = model(input_batch, hidden, output_batch)
        output = F.softmax(output, dim=2)  # Aplicar softmax para obtener probabilidades
        sequences = [[list(), 1.0]]  # (sequence, score)

        for row in output.squeeze(1):  # Asegurar que se procesa la salida correcta
            all_candidates = list()
            for seq, score in sequences:
                for j in range(len(row)):
                    candidate = [seq + [j], score * row[j].item()]  # Multiplicar las probabilidades, no sumar logaritmos negativos
                    all_candidates.append(candidate)
            # Ordenar todos los candidatos por puntuación de manera descendente (probabilidad más alta primero)
            ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
            sequences = ordered[:beam_width]

        best_sequence = sequences[0][0]
        return torch.tensor(best_sequence, dtype=torch.long).view(1, -1, 1)


    def random_sample_with_temperature(output, temperature=1.0):
        output = output.div(temperature).exp()
        probs = F.softmax(output, dim=-1)
        return torch.multinomial(probs.view(-1, probs.size(-1)), 1).view(output.size(0), output.size(1), -1)

    def decode_with_temperature(input_batch, hidden, output_batch, temperature=1.0):
        output = model(input_batch, hidden, output_batch)
        sampled_output = random_sample_with_temperature(output, temperature)
        return sampled_output

    def translate(word, strategy='greedy', beam_width=3, temperature=1.0):
        input_batch, output_batch = make_testbatch(word)
        hidden = torch.zeros(2, 1, n_hidden) # cambiar a 2 capas
    
        if strategy == 'greedy':
            predict = greedy_decode(input_batch, hidden, output_batch)
        elif strategy == 'beam_search':
            predict = beam_search_decode(input_batch, hidden, output_batch, beam_width)
        elif strategy == 'temperature':
            predict = decode_with_temperature(input_batch, hidden, output_batch, temperature)
    
        decoded = [char_arr[i] for i in predict.squeeze()]
        if 'E' in decoded:
            end = decoded.index('E')
            translated = ''.join(decoded[:end])
        else:
            translated = ''.join(decoded)

        return translated.replace('P', '')

    words = ['man', 'mans', 'king', 'black', 'upp']
    
    print('Comparativa de decodificación:\n')
    
    for word in words:
        print(f'Palabra: {word}')
        print(f'Greedy: {translate(word, strategy="greedy")}')
        print(f'Beam Search: {translate(word, strategy="beam_search", beam_width=3)}')
        print(f'Temperatura (T=1.0): {translate(word, strategy="temperature", temperature=1.0)}')
        print('---------------------------------')


### Ejercicios

1 .Implementa un mecanismo de atención en el modelo Seq2Seq para mejorar la traducción.

Instrucciones:

- Añade una capa de atención al modelo Seq2Seq.
- Modifica el método forward para incorporar la atención.
- Entrena el modelo y compara el rendimiento con el modelo original.

Pistas:

- Utiliza la clase nn.Linear para calcular los pesos de atención.
- Multiplica los pesos de atención con los estados ocultos del codificador para obtener el contexto.
- Concatena el contexto con la entrada del decodificador en cada paso de tiempo.

2 . Compara el rendimiento de las estrategias de decodificación Greedy, Beam Search y Temperatura en diferentes configuraciones de entrenamiento.

Instrucciones:

- Entrena el modelo con diferentes tamaños de conjunto de datos y configuraciones de hiperparámetros (por ejemplo, diferentes tamaños de hidden_size y num_layers).
- Aplica las tres estrategias de decodificación a cada modelo entrenado.
- Evalúa y compara la calidad de las traducciones utilizando métricas como la precisión y el BLEU score.

Pistas:

- Usa conjuntos de datos grandes y pequeños para ver cómo cambia el rendimiento.
- Experimenta con diferentes beam_width y temperatures.

3 . Implementa una variante de Beam Search que penalice secuencias más largas para evitar repeticiones innecesarias.

Instrucciones:

- Modifica la función beam_search_decode para incluir una penalización de longitud.
- Ajusta el cálculo de las puntuaciones de las secuencias para penalizar las secuencias más largas.
- Compara los resultados con el Beam Search estándar.

Pistas:

- Multiplica la puntuación de cada secuencia por una función de penalización basada en su longitud.
- Puedes usar una función de penalización lineal o exponencial.

4 . Implementa una estrategia de decodificación por temperatura que ajuste dinámicamente la temperatura en cada paso de tiempo.

Instrucciones:

- Modifica la función decode_with_temperature para ajustar la temperatura en cada paso de tiempo.
- Implementa una función que disminuya la temperatura a medida que avanza la decodificación, incentivando exploración al principio y explotación al final.
- Evalúa el impacto de la temperatura dinámica en la calidad de las traducciones.

Pistas:

- Usa una función de decremento lineal o exponencial para la temperatura.
- Compara los resultados con una temperatura fija.

5 . Analiza la complejidad computacional y el rendimiento de las diferentes estrategias de decodificación.

Instrucciones:

- Mide el tiempo de ejecución de las estrategias Greedy, Beam Search y Temperatura para diferentes tamaños de vocabulario y longitudes de secuencia.
- Analiza cómo cambia la complejidad computacional con respecto a beam_width y temperature.
- Discute los trade-offs entre la calidad de la traducción y el tiempo de ejecución.

Pistas:

- Usa la biblioteca time para medir el tiempo de ejecución.
- Realiza pruebas con diferentes configuraciones y grafica los resultados.


6 .Mejora la generalización del modelo incorporando técnicas de regularización y dropout.

Instrucciones:

- Añade capas de dropout adicionales al modelo Seq2Seq.
- Implementa técnicas de regularización como L2.
- Entrena el modelo con estas técnicas y compara el rendimiento con el modelo original.

Pistas:

- Usa nn.Dropout en las capas RNN y totalmente conectadas.
- Ajusta los hiperparámetros de regularización y dropout para encontrar la configuración óptima

In [ ]:
## Tus respuestas